In [ ]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import gzip
import random
import scipy
#import tensorflow as tf
from collections import defaultdict
from implicit import bpr
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
from implicit.cpu.lmf import LogisticMatrixFactorization
from implicit.cpu.als import AlternatingLeastSquares

In [ ]:
def readJSON(path):
    #f = gzip.open(path, 'rt')
    f = open(path)
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [ ]:
data = []
for l in readJSON("train.json"):
    data.append(l)

In [ ]:
random.shuffle(data)

In [ ]:
train = data[:165000]
valid = data[165000:]

In [ ]:
userIDs,itemIDs = {},{}

for u,i,_ in train:
    #u,i = d['user_id'],d['book_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

In [ ]:
nUsers,nItems

In [ ]:
Xui = scipy.sparse.lil_matrix((nUsers, nItems))
for u,g,_ in train:
    Xui[userIDs[u],itemIDs[g]] = 1
    
Xui_csr = scipy.sparse.csr_matrix(Xui)

In [ ]:
model = bpr.BayesianPersonalizedRanking(factors = 10)

In [ ]:
model.fit(Xui_csr)

In [ ]:
ids, scores = model.recommend(0, Xui_csr[0],N=20)
# print(ids)
# print(scores)

In [ ]:
# usersPerGame = defaultdict(list)
# gamesPerUser = defaultdict(list)
# for user,game,_ in data:
#     usersPerGame[game].append(user)
#      gamesPerUser[user].append(game)
# games = list(usersPerGame.keys())
# users = list(gamesPerUser.keys())

In [ ]:
newValid = []
falseSamples = defaultdict(list)
for u,g,_ in valid:
    newValid.append([u,g,1])
    res = random.choice([ele for ele in games if ele not in gamesPerUser[u]+falseSamples[u]])
    falseSamples[u].append(res)
    newValid.append([u,res,0])

In [ ]:
model = LogisticMatrixFactorization(factors = 10,regularization = 10)

In [ ]:
model.fit(Xui_csr)

In [ ]:
model.recommend(0, Xui_csr[0],N=20)

In [ ]:
features = []
y = []
for j in range(len(newTrain)):
    u = newTrain[j][0]
    i = newTrain[j][1]
    l = newTrain[j][2]
    y.append(l)
    temp = [1]
#     uid = userIDs[u]
#     ids, scores = model.recommend(uid, Xui_csr[uid],N=nItems)
    #print(scores)
#     if (itemIDs[i] in list(ids)) and (scores[list(ids).index(itemIDs[i])]>=0):
    #if (scores[list(ids).index(itemIDs[i])]>=0):
#         pred.append(1)
#     else:
#         pred.append(0)
    a = 0
    if u in userIDs:
        uid = userIDs[u] 
        ids, scores = model.recommend(uid, Xui_csr[uid],N=nItems)
        a = scores[list(ids).index(itemIDs[i])]
        
    temp.append(a)
    temp.append(max_sims[j])
    features.append(temp)
# cor = 0
# tot = 0
# for i in range(len(y)):
#     if(y[i]==pred[i]):
#         cor+=1
#     tot+=1
# cor/tot

In [ ]:
model1 = linear_model.LogisticRegression(C=1,max_iter=500)
model1.fit(features,y)
pred = model1.predict(features)
accuracy = sum(pred==y)/len(y)
accuracy

In [ ]:
features = []
y = []
for j in range(len(newValid)):
    u = newValid[j][0]
    i = newValid[j][1]
    l = newValid[j][2]
    y.append(l)
    temp = [1]
    if u in userIDs:
        uid = userIDs[u] 
        ids, scores = model.recommend(uid, Xui_csr[uid],N=nItems)
        a = scores[list(ids).index(itemIDs[i])]
        
    temp.append(a)
    temp.append(max_sims_valid[j])
    features.append(temp)

In [ ]:
pred = model1.predict(features)
accuracy = sum(pred==y)/len(y)
accuracy

In [ ]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    gamesByUser = gamesPerUserTrain[u]
    users = usersPerGameTrain[g]
    max_sim = 0
    
    for g_ in gamesByUser:
        if (g_==g): continue
        sim = Jaccard(users, usersPerGameTrain[g_])
        max_sim = max(max_sim,sim)
    
    a = 0
    if u in userIDs:
        uid = userIDs[u] 
        ids, scores = model.recommend(uid, Xui_csr[uid],N=nItems)
        a = scores[list(ids).index(itemIDs[i])]
    
    pred = model1.predict([[1,a,max_sim]])[0]
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

In [ ]:
usersPerGameTrain = defaultdict(set)
gamesPerUserTrain = defaultdict(set)
for user,game,_ in train:
    usersPerGameTrain[game].add(user)
    gamesPerUserTrain[user].add(game)
gamesTrain = list(usersPerGameTrain.keys())
usersTrain = list(gamesPerUserTrain.keys())

In [ ]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
max_sims_valid = []
for i in range(len(newValid)):
    u = newValid[i][0]
    g = newValid[i][1]
    gamesByUser = gamesPerUserTrain[u]
    users = usersPerGameTrain[g]
    max_sim = 0
    
    for g_ in gamesByUser:
        if (g_==g): continue
        sim = Jaccard(users, usersPerGameTrain[g_])
        max_sim = max(max_sim,sim)
    max_sims_valid.append(max_sim)

In [ ]:
newTrain = []
falseSamples = defaultdict(list)
for u,g,_ in train:
    newTrain.append([u,g,1])
    res = random.choice([ele for ele in games if ele not in gamesPerUser[u]+falseSamples[u]])
    falseSamples[u].append(res)
    newTrain.append([u,res,0])

In [ ]:
max_sims = []
for i in range(len(newTrain)):
    u = newTrain[i][0]
    g = newTrain[i][1]
    gamesByUser = gamesPerUserTrain[u]
    users = usersPerGameTrain[g]
    max_sim = 0
    
    for g_ in gamesByUser:
        if (g_==g): continue
        sim = Jaccard(users, usersPerGameTrain[g_])
        max_sim = max(max_sim,sim)
    max_sims.append(max_sim)